### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import logging
import datetime, time
import platform
import itertools
# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt

# OpenTNSim
import opentnsim

# Used for mathematical functions
import math             
import tqdm
# Used for making the graph to visualize our problem
import networkx as nx   

import plotly.express as px
from plotly.subplots import make_subplots

#logging.basicConfig(level=logging.DEBUG) #turn on all the debug messages
logging.basicConfig(level=logging.INFO)# turn off all the debug messages


### Create vessel - add VesselProperties and ConsumesEnergy mixins


In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel',
    "route": None,
    "geometry": None,
    "V_g_ave": None,  # m/s
    "type": None,
    "B": 9.5,
    "L": 155,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":None, # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_tot_given_profile":False,
    "P_installed": 780, # kW  
    "P_tot_given": True, # kW
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": True,
    "sailing_upstream":False,
    "wind_influence": False, # if consider wind influence, set to True; otherwise set to False.
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 3.0 ,
    "C_B":0.9, 
    "C_year": 1961,
}             



### arrange input table

In [4]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
depth = [7.5, 3.7744,7.7178,6.5,10.6,6.5531,8.1001,5.8238,8.9354,4.9192,7.7,9.8,5.5799]
power_applied_down = [515,408,230,449,235,179,469,633,327,153,296,82,235,163,194]
power_applied_up = [633,480]

In [5]:
# prepare the work to be done by creating a list of all combinations
work_up = list(itertools.product(stretch, depth, width,power_applied_up))

# prepare a list of dictionaries for pandas
rows_up = []
for item in work_up:
    row_up = {"stretch": item[0],"depth": item[1],"width":item[2], "power_applied_up":item[3]}
    rows_up.append(row_up)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_up_df = pd.DataFrame(rows_up)
work_up_df.tail(100)

,stretch,depth,width,power_applied_up
290,s12,7.7178,150,633
291,s12,7.7178,150,480
292,s12,6.5000,150,633
293,s12,6.5000,150,480
294,s12,10.6000,150,633
...,...,...,...,...
385,s15,7.7000,150,480
386,s15,9.8000,150,633
387,s15,9.8000,150,480
388,s15,5.5799,150,633


### Run simulation


In [6]:
results_up = []

for i, row_up in tqdm.tqdm(work_up_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_up['power_applied_up']
    
    stretch = row_up['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    V_w_up = opentnsim.strategy.power2v(vessel, h_0=row_up['depth'],power_applied=row_up['power_applied_up'], upperbound=5)
    # V_w_down = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_down'], upperbound=5)
    
    result_up ={}
    result_up.update(row_up)
    
    result_up['V_w_up (m/s)'] = V_w_up    
    result_up['V_w_up (km/h)'] = V_w_up * 3.6
    # result_up['V_w_down (m/s)'] = V_w_down    
    # result_up['V_w_down (km/h)'] = V_w_down * 3.6
    results_up.append(result_up)

In [7]:
results_up_df = pd.DataFrame(results_up)
results_up_df

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
0,s1,7.5000,150,633,2.925749,10.532695
1,s1,7.5000,150,480,2.733487,9.840553
2,s1,3.7744,150,633,2.741342,9.868832
3,s1,3.7744,150,480,2.578185,9.281467
4,s1,7.7178,150,633,2.925956,10.533440
...,...,...,...,...,...,...
385,s15,7.7000,150,480,2.733683,9.841259
386,s15,9.8000,150,633,3.413744,12.289479
387,s15,9.8000,150,480,3.075303,11.071091
388,s15,5.5799,150,633,2.940784,10.586823


In [21]:
V1_up = results_up_df.query('stretch == "s1" & depth == 7.5 & power_applied_up == 633')
V2_up = results_up_df.query('stretch == "s2" & depth == 3.7744 & power_applied_up == 633')
V3_up = results_up_df.query('stretch == "s3" & depth == 3.7744 & power_applied_up == 633')
V4_up = results_up_df.query('stretch == "s4" & depth == 7.7178 & power_applied_up == 633')
V5_up = results_up_df.query('stretch == "s5" & depth == 7.5 & power_applied_up == 633')
V6_up = results_up_df.query('stretch == "s6" & depth == 6.5 & power_applied_up == 633')
V7_up = results_up_df.query('stretch == "s7" & depth == 10.6 & power_applied_up == 633')
V8_up = results_up_df.query('stretch == "s8" & depth == 6.5531 & power_applied_up == 633')
V9_up = results_up_df.query('stretch == "s9" & depth == 8.1001 & power_applied_up == 633')
V10_up = results_up_df.query('stretch == "s10" & depth == 5.8238 & power_applied_up == 633')
V11_up = results_up_df.query('stretch == "s11" & depth == 8.9354 & power_applied_up == 633')
V12_up = results_up_df.query('stretch == "s12" & depth == 4.9192 & power_applied_up == 633')
V13_up = results_up_df.query('stretch == "s13" & depth == 7.7 & power_applied_up == 633')
V14_up = results_up_df.query('stretch == "s14" & depth == 9.8 & power_applied_up == 480')
V15_up = results_up_df.query('stretch == "s15" & depth == 5.5799 & power_applied_up == 633')
V_up = pd.concat([V1_up,V2_up,V3_up,V4_up,V5_up,V6_up,V7_up,V8_up,V9_up,V10_up,V11_up,V12_up,V13_up,V14_up,V15_up])


V_up= V_up.drop_duplicates()
V_up.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_up

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h)
1,s1,7.5000,150,633,2.925749,10.532695
2,s2,3.7744,150,633,2.741342,9.868832
3,s3,3.7744,150,633,2.741342,9.868832
4,s4,7.7178,150,633,2.925956,10.533440
5,s5,7.5000,150,633,2.925749,10.532695
6,s6,6.5000,150,633,2.942625,10.593451
7,s7,10.6000,150,633,3.278881,11.803973
8,s8,6.5531,150,633,2.942706,10.593743
9,s9,8.1001,150,633,2.908117,10.469220
10,s10,5.8238,150,633,2.941369,10.588928


In [9]:
stretch = ["s1","s2","s3","s4","s5","s6","s7","s8",
          "s9","s10","s11","s12","s13","s14","s15"]
width = [150]
depth = [7.5, 3.7744,7.7178,6.5,10.6,6.5531,8.1001,5.8238,8.9354,4.9192,7.7,9.8,5.5799]
power_applied_down = [515,408,230,449,235,179,469,633,327,153,296,82,235,163,194]
power_applied_up = [633,480]

In [10]:
# prepare the work to be done by creating a list of all combinations
work_down = list(itertools.product(stretch, depth, width, power_applied_down))

# prepare a list of dictionaries for pandas
rows_down = []
for item in work_down:
    row_down = {"stretch": item[0], "depth": item[1],"width":item[2], "power_applied_down": item[3]}
    rows_down.append(row_down)

    # these are all the simulations that we want to run

# convert them to dataframe, so that we can apply a function and monitor progress
work_down_df = pd.DataFrame(rows_down)
work_down_df.tail(100)

,stretch,depth,width,power_applied_down
2825,s15,8.1001,150,179
2826,s15,8.1001,150,469
2827,s15,8.1001,150,633
2828,s15,8.1001,150,327
2829,s15,8.1001,150,153
...,...,...,...,...
2920,s15,5.5799,150,296
2921,s15,5.5799,150,82
2922,s15,5.5799,150,235
2923,s15,5.5799,150,163


In [11]:
results_down = []

for i, row_down in tqdm.tqdm(work_down_df.iterrows(),disable=True):

    # get vessel 
    data_vessel_i = data_vessel.copy()
    vessel = TransportResource(**data_vessel_i)
    vessel.P_tot_given = row_down['power_applied_down']
    stretch = row_down['stretch']
    # estimate 'grounding speed' as a useful upperbound
    # upperbound, selected, results_df = opentnsim.strategy.get_upperbound_for_power2v(vessel, width=row['width'], depth=row['depth'], margin=0.3,bounds=(0, 20))
    # print(upperbound)
    # calculate the velocity that belongs to the T_strategy (while leaving the margin)
    # V_w_up = opentnsim.strategy.power2v(vessel, h_0=row['depth'],power_applied=row['power_applied_up'], upperbound=5)
    V_w_down = opentnsim.strategy.power2v(vessel, h_0=row_down['depth'],power_applied=row_down['power_applied_down'], upperbound=5)
    
    result_down ={}
    result_down.update(row_down)
    
    # result['V_w_up (m/s)'] = V_w_up    
    # result['V_w_up (km/h)'] = V_w_up * 3.6
    result_down['V_w_down (m/s)'] = V_w_down    
    result_down['V_w_down (km/h)'] = V_w_down * 3.6
    results_down.append(result_down)

In [12]:
results_down_df = pd.DataFrame(results_down)
results_down_df

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
0,s1,7.5000,150,515,2.776431,9.995151
1,s1,7.5000,150,408,2.637353,9.494472
2,s1,7.5000,150,230,2.297534,8.271122
3,s1,7.5000,150,449,2.693497,9.696588
4,s1,7.5000,150,235,2.308340,8.310022
...,...,...,...,...,...,...
2920,s15,5.5799,150,296,2.470481,8.893731
2921,s15,5.5799,150,82,1.817147,6.541729
2922,s15,5.5799,150,235,2.334644,8.404718
2923,s15,5.5799,150,163,2.127956,7.660642


In [20]:
V1_down = results_down_df.query('stretch == "s1" & depth == 7.5 & power_applied_down == 515')
V2_down = results_down_df.query('stretch == "s2" & depth == 3.7744 & power_applied_down == 408')
V3_down = results_down_df.query('stretch == "s3" & depth == 3.7744 & power_applied_down == 230')
V4_down = results_down_df.query('stretch == "s4" & depth == 7.7178 & power_applied_down == 449')
V5_down = results_down_df.query('stretch == "s5" & depth == 7.5 & power_applied_down == 235')
V6_down = results_down_df.query('stretch == "s6" & depth == 6.5 & power_applied_down == 179')
V7_down = results_down_df.query('stretch == "s7" & depth == 10.6 & power_applied_down == 469')
V8_down = results_down_df.query('stretch == "s8" & depth == 6.5531 & power_applied_down == 633')
V9_down = results_down_df.query('stretch == "s9" & depth == 8.1001 & power_applied_down == 327')
V10_down = results_down_df.query('stretch == "s10" & depth == 5.8238 & power_applied_down == 153')
V11_down = results_down_df.query('stretch == "s11" & depth == 8.9354 & power_applied_down == 296')
V12_down = results_down_df.query('stretch == "s12" & depth == 4.9192 & power_applied_down == 82')
V13_down = results_down_df.query('stretch == "s13" & depth == 7.7 & power_applied_down == 235')
V14_down = results_down_df.query('stretch == "s14" & depth == 9.8 & power_applied_down == 163')
V15_down = results_down_df.query('stretch == "s15" & depth == 5.5799 & power_applied_down == 194')
V_down = pd.concat([V1_down,V2_down,V3_down,V4_down,V5_down,V6_down,
                    V7_down,V8_down,V9_down,V10_down,V11_down,V12_down,V13_down,V14_down,V15_down])
V_down= V_down.drop_duplicates()
V_down.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
V_down

,stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h)
1,s1,7.5000,150,515,2.776431,9.995151
2,s2,3.7744,150,408,2.487455,8.954839
3,s3,3.7744,150,230,2.304799,8.297276
4,s4,7.7178,150,449,2.693711,9.697359
5,s5,7.5000,150,235,2.308340,8.310022
6,s6,6.5000,150,179,2.173283,7.823819
7,s7,10.6000,150,469,3.059343,11.013636
8,s8,6.5531,150,633,2.942706,10.593743
9,s9,8.1001,150,327,2.481559,8.933611
10,s10,5.8238,150,153,2.097529,7.551106


### get vessel velocity to the ground: upstream, downstream

In [30]:
current_speeds = pd.DataFrame([-1.7438, -3.0534, -4.49796, -2.0406,
                           -3.4768 , -4.0886, -2.0002, -1.1236,
                           -2.9767, -5.1473, -3.0532, -6.384,
                           -3.4064 , -1.6336, -4.1819], )
current_speeds.index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
current_speeds.columns =['current_speeds (km/h)']

V_df = pd.concat([V_up,V_down,current_speeds], axis = 1)
V_df['V_g_upstream (km/h)']= V_df['V_w_up (km/h)']+V_df['current_speeds (km/h)']
V_df['V_g_downstream (km/h)']= V_df['V_w_down (km/h)']-V_df['current_speeds (km/h)']

V_df

,stretch,depth,width,power_applied_up,V_w_up (m/s),V_w_up (km/h),stretch,depth,width,power_applied_down,V_w_down (m/s),V_w_down (km/h),current_speeds (km/h),V_g_upstream (km/h),V_g_downstream (km/h)
1,s1,7.5000,150,633,2.925749,10.532695,s1,7.5000,150,515,2.776431,9.995151,-1.74380,8.788895,11.738951
2,s2,3.7744,150,633,2.741342,9.868832,s2,3.7744,150,408,2.487455,8.954839,-3.05340,6.815432,12.008239
3,s3,3.7744,150,633,2.741342,9.868832,s3,3.7744,150,230,2.304799,8.297276,-4.49796,5.370872,12.795236
4,s4,7.7178,150,633,2.925956,10.533440,s4,7.7178,150,449,2.693711,9.697359,-2.04060,8.492840,11.737959
5,s5,7.5000,150,633,2.925749,10.532695,s5,7.5000,150,235,2.308340,8.310022,-3.47680,7.055895,11.786822
6,s6,6.5000,150,633,2.942625,10.593451,s6,6.5000,150,179,2.173283,7.823819,-4.08860,6.504851,11.912419
7,s7,10.6000,150,633,3.278881,11.803973,s7,10.6000,150,469,3.059343,11.013636,-2.00020,9.803773,13.013836
8,s8,6.5531,150,633,2.942706,10.593743,s8,6.5531,150,633,2.942706,10.593743,-1.12360,9.470143,11.717343
9,s9,8.1001,150,633,2.908117,10.469220,s9,8.1001,150,327,2.481559,8.933611,-2.97670,7.492520,11.910311
10,s10,5.8238,150,633,2.941369,10.588928,s10,5.8238,150,153,2.097529,7.551106,-5.14730,5.441628,12.698406


### get vessel sailing duration: upstream, downstream, roundtrip